In [37]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [15]:
#load data

full_df = pd.read_csv('raw_data/kickstarter_projects.csv', low_memory=False)
full_df.head()

,id,kickstarter_id,name,slug,blurb,creator,main_category,sub_category,launched_at,deadline,...,became_underdog_candidate_at,underdog_chosen_by_user_id,photo,photo_medium,web_url,currency,currency_symbol,has_accurate_category,created_at,updated_at
0,1,454086636,The Cargo Sling - Don't Lose Your Load on the ...,the-cargo-sling-dont-lose-your-load-on-the-road,Trouble securing those long wood or plastic bo...,Jim DelPrete,Design,Product Design,6/2/2013 0:12,7/2/2013 0:12,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/510/419/1...,https://ksr-ugc.imgix.net/assets/011/510/419/1...,https://www.kickstarter.com/projects/576540451...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46
1,11,1227342522,Starlite objects project.,starlite-objects-project,"\Flat packed objects up for grabs\"".""",Chijioke Aguh,Design,Product Design,6/2/2013 0:13,6/22/2013 0:13,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/522/289/9...,https://ksr-ugc.imgix.net/assets/011/522/289/9...,https://www.kickstarter.com/projects/203310430...,GBP,£,1,6/8/2013 23:44,3/9/2017 6:46
2,21,1061247348,Fall and Recover: A Refugee Dance Film,fall-and-recover-a-refugee-dance-project,War torture survivors from Africa and The Midd...,Jason Somma + Erinn Clancy,Film & Video,Film & Video,6/2/2013 0:52,8/1/2013 0:52,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/464/192/b...,https://ksr-ugc.imgix.net/assets/011/464/192/b...,https://www.kickstarter.com/projects/73848066/...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46
3,31,1090116816,After 68: The Rise and Fall of The Ambassador ...,after-68,A gripping exposé on the dynamic history and c...,Camilo Silva,Film & Video,Documentary,6/2/2013 2:00,7/24/2013 2:00,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/457/736/8...,https://ksr-ugc.imgix.net/assets/011/457/736/8...,https://www.kickstarter.com/projects/camilosil...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46
4,41,636501097,TagPodz: A Pet Tag Silencer for Dogs (Canceled),tagpodz-a-stylish-pet-tag-holder-for-dogs,TagPodz is redefining how we attach pet tags t...,Matt & Carol,Design,Product Design,6/2/2013 2:30,8/1/2013 2:30,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/464/026/5...,https://ksr-ugc.imgix.net/assets/011/464/026/5...,https://www.kickstarter.com/projects/tagpodz/t...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46


In [29]:
#remove ongoing campaigns and misformatted rows

success_df = full_df[full_df.state == 'successful']
failure_df = full_df[full_df.state == 'failed']
canceled_df = full_df[full_df.state == 'canceled']
dfs = [success_df, failure_df, canceled_df]
kickstarter_df = pd.concat(dfs)
kickstarter_df.head()

,id,kickstarter_id,name,slug,blurb,creator,main_category,sub_category,launched_at,deadline,...,became_underdog_candidate_at,underdog_chosen_by_user_id,photo,photo_medium,web_url,currency,currency_symbol,has_accurate_category,created_at,updated_at
2,21,1061247348,Fall and Recover: A Refugee Dance Film,fall-and-recover-a-refugee-dance-project,War torture survivors from Africa and The Midd...,Jason Somma + Erinn Clancy,Film & Video,Film & Video,6/2/2013 0:52,8/1/2013 0:52,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/464/192/b...,https://ksr-ugc.imgix.net/assets/011/464/192/b...,https://www.kickstarter.com/projects/73848066/...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46
5,51,619552907,The History of Iran Podcast,the-history-of-iran-podcast,This is a podcast covering the history of Iran...,Khodadad Rezakhani (deleted),Publishing,Nonfiction,6/2/2013 2:37,7/2/2013 2:37,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/514/197/2...,https://ksr-ugc.imgix.net/assets/011/514/197/2...,https://www.kickstarter.com/projects/124208487...,GBP,£,1,6/8/2013 23:44,3/9/2017 6:46
7,71,1487530880,Harvest,harvest-0,HARVEST is a Graphic Adventure similar to Snat...,GondeFire Productions,Games,Video Games,6/2/2013 3:23,7/2/2013 3:23,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/500/160/5...,https://ksr-ugc.imgix.net/assets/011/500/160/5...,https://www.kickstarter.com/projects/184029310...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46
8,81,1778766009,colby dix - the 'five',colby-dix-the-five-ep,after years of making noise and helping others...,Colby Dix,Music,Indie Rock,6/2/2013 3:45,6/30/2013 16:00,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/527/176/b...,https://ksr-ugc.imgix.net/assets/011/527/176/b...,https://www.kickstarter.com/projects/colbydix/...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46
13,131,8923343,Driftwood,driftwood,A story of two hikers who cross paths on the t...,Heidi Gray,Film & Video,Shorts,6/2/2013 13:01,7/2/2013 13:01,...,NaN,NaN,https://ksr-ugc.imgix.net/assets/011/525/147/d...,https://ksr-ugc.imgix.net/assets/011/525/147/d...,https://www.kickstarter.com/projects/177160458...,USD,$,1,6/8/2013 23:44,3/9/2017 6:46


In [34]:
#extract only features of interest

features_of_interest = pd.DataFrame({
    "kickstarter_id":kickstarter_df["kickstarter_id"],
    "name":kickstarter_df["name"],
    "slug":kickstarter_df["slug"],
    "blurb":kickstarter_df["blurb"],
    "main_category":kickstarter_df["main_category"],
    "sub_category":kickstarter_df["sub_category"],
    "launch_date":kickstarter_df["launched_at"],
    "deadline_date":kickstarter_df["deadline"],
    "result":kickstarter_df["state"],
    "goal":kickstarter_df["goal"],
    "pledged":kickstarter_df["pledged"],
    "pct_funded":kickstarter_df["percentage_funded"],
    "backer_count":kickstarter_df["backers_count"],
    "currency":kickstarter_df["currency"]
})

features_of_interest.head()

,backer_count,blurb,currency,deadline_date,goal,kickstarter_id,launch_date,main_category,name,pct_funded,pledged,result,slug,sub_category
2,92,War torture survivors from Africa and The Midd...,USD,8/1/2013 0:52,8000,1061247348,6/2/2013 0:52,Film & Video,Fall and Recover: A Refugee Dance Film,1.06875,8550,successful,fall-and-recover-a-refugee-dance-project,Film & Video
5,12,This is a podcast covering the history of Iran...,GBP,7/2/2013 2:37,450,619552907,6/2/2013 2:37,Publishing,The History of Iran Podcast,1.11333,501,successful,the-history-of-iran-podcast,Nonfiction
7,394,HARVEST is a Graphic Adventure similar to Snat...,USD,7/2/2013 3:23,10000,1487530880,6/2/2013 3:23,Games,Harvest,1.1879,11878.99,successful,harvest-0,Video Games
8,97,after years of making noise and helping others...,USD,6/30/2013 16:00,2000,1778766009,6/2/2013 3:45,Music,colby dix - the 'five',3.2005,6400.99,successful,colby-dix-the-five-ep,Indie Rock
13,23,A story of two hikers who cross paths on the t...,USD,7/2/2013 13:01,2000,8923343,6/2/2013 13:01,Film & Video,Driftwood,1.06,2120,successful,driftwood,Shorts


In [43]:
blurbs = features_of_interest["blurb"]
blurb_lengths = []
blurb_sentiments = []

for blurb in blurbs:
    try:
        blurb_length = len(blurb)
        sentiment = analyzer.polarity_scores(blurb)['compound']
    except TypeError:
        blurb_length = 0
        sentiment = 0
    blurb_sentiments.append(sentiment)
    blurb_lengths.append(blurb_length)

features_of_interest['blurb_length'] = blurb_lengths
features_of_interest['blurb_sentiment'] = blurb_sentiments

features_of_interest.head()

,backer_count,blurb,currency,deadline_date,goal,kickstarter_id,launch_date,main_category,name,pct_funded,pledged,result,slug,sub_category,blurb_length,blurb_sentiment
2,92,War torture survivors from Africa and The Midd...,USD,8/1/2013 0:52,8000,1061247348,6/2/2013 0:52,Film & Video,Fall and Recover: A Refugee Dance Film,1.06875,8550,successful,fall-and-recover-a-refugee-dance-project,Film & Video,132,-0.9100
5,12,This is a podcast covering the history of Iran...,GBP,7/2/2013 2:37,450,619552907,6/2/2013 2:37,Publishing,The History of Iran Podcast,1.11333,501,successful,the-history-of-iran-podcast,Nonfiction,118,0.0000
7,394,HARVEST is a Graphic Adventure similar to Snat...,USD,7/2/2013 3:23,10000,1487530880,6/2/2013 3:23,Games,Harvest,1.1879,11878.99,successful,harvest-0,Video Games,133,0.5719
8,97,after years of making noise and helping others...,USD,6/30/2013 16:00,2000,1778766009,6/2/2013 3:45,Music,colby dix - the 'five',3.2005,6400.99,successful,colby-dix-the-five-ep,Indie Rock,131,0.5994
13,23,A story of two hikers who cross paths on the t...,USD,7/2/2013 13:01,2000,8923343,6/2/2013 13:01,Film & Video,Driftwood,1.06,2120,successful,driftwood,Shorts,96,0.0000
